# Execução do modelo

## Instalação das dependências

In [ ]:
!pip install unsloth huggingface_hub

: 

In [ ]:
from os import environ

import torch

for i in range(torch.cuda.device_count()):
    total_memory = torch.cuda.get_device_properties(i).total_memory
    reserved_memory = torch.cuda.memory_reserved(i)
    allocated_memory = torch.cuda.memory_allocated(i)
    free_memory = reserved_memory - allocated_memory

    print(f'[GPU {i}] - Total: {total_memory}, Reserved: {reserved_memory}, Allocated: {allocated_memory}, Free: {free_memory}')

device_id = int(input('Select a device: '))

torch.cuda.set_device(device_id)

print(f'Current GPU: {torch.cuda.current_device()}')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/erfer/Documents/UFSC/INE5443_RP/Code/Computer-Vision/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  File "/tmp/ipykernel_32840/1584800579.py", line 1, in <module>
    from unsloth import FastVisionModel
ModuleNotFoundError: No module named 'unsloth'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/erfer/Documents/UFSC/INE5443_RP/Code/Computer-Vision/.venv/lib/python3.12/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/erfer/Documents/UFSC/INE5443_RP/Code/Computer-Vision/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 2168, in showtraceback
  File "/home/erfer/Documents/UFSC/INE5443_RP/Code/Co

In [ ]:
from os import getenv, mkdir
from os.path import exists
from dotenv import load_dotenv
from huggingface_hub import login

hf_token = None

if not exists('../.env'):
    hf_token = input('Enter your Hugging Face token: ')
else:
    load_dotenv(dotenv_path='../.env')
    hf_token = getenv('HF_TOKEN')

assert hf_token is not None, 'Invalid Hugging Face token'

login(token=hf_token)

In [ ]:
from datasets import load_dataset

dataset = load_dataset('marmal88/skin_cancer', split='test')

In [ ]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    'unsloth/Llama-3.2-11B-Vision-Instruct',
    load_in_4bit = True,
    use_gradient_checkpointing = 'unsloth',
)

In [ ]:
FastVisionModel.for_inference(model)

image = dataset[0]['image']

messages = [
    {
        'role': 'user',
        'content': [
            {'type': 'image'},
            {'type': 'text', 'text': 'You are a model trained to identify skin lesions. Identify the skin disease in the image.'}
        ]
    }
]

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)

inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = 'pt',
).to('cuda')

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)